In [1]:
#1. 가상 데이터 200개 생성

import pandas as pd
import random
from faker import Faker

# Faker 객체 생성
fake = Faker("ko_KR")

# 보험사 이름 리스트
insurance_companies = [
    "삼성화재", "현대해상", "DB손해보험", "KB손해보험", "롯데손해보험", "한화손해보험",
    "메리츠화재", "흥국화재", "NH농협생명", "교보생명", "삼성생명", "한화생명"
]

# 보험 상품 종류 리스트
insurance_products = [
    "연금보험", "종신보험", "변액연금보험", "실비보험", "어린이보험", "치아보험", "간병보험", "건강보험", "자동차보험"
]

# 보험사와 상품명을 조합하는 함수
def generate_random_product():
    company = random.choice(insurance_companies)
    product = random.choice(insurance_products)
    return f"{company} {product}"

# 상담유형 매핑
type_mapping = {
    "01": "상품불만",
    "02": "설계사불만",
    "03": "보험금지급불만",
    "04": "보험료해지",
    "05": "보장내용",
    "06": "기타"
}

# 점수 및 부정 여부 기준 계산 함수
def calculate_score(negative_count):
    if negative_count <= 2:
        return "긍정"
    elif 3 <= negative_count <= 5:
        return "중립"
    elif 6 <= negative_count < 10:
        return "부정"
    else:
        return "강한 불만"

# 대화 내용 요약 생성 함수
def summarize_call_content(content):
    summary_start = content[:50]  # 처음 50자 추출
    return f"{summary_start}..." if len(content) > 50 else content

# 대화 내용 한국어 예시 생성 함수
def generate_korean_call_content():
    return random.choice([
        "보험료가 너무 비싸요. 약정과 다릅니다.",
        "설계사가 계약 내용을 잘 설명하지 않았어요.",
        "보험금을 청구했는데 지급이 지연되고 있어요.",
        "보험료를 해지하고 싶은데 절차가 복잡합니다.",
        "보장 내용이 생각보다 제한적이에요.",
        "기타 불만 사항이 있습니다."
    ])

# 가상 데이터 생성
data = [
    {
        "customer_id": fake.unique.random_int(min=1000, max=9999),
        "customer_name": fake.name(),
        "product_name": generate_random_product(),  # 조합된 보험 상품명
        "call_id": fake.uuid4(),
        "agent_id": fake.random_int(min=100, max=999),
        "call_date": fake.date_this_year(),
        "call_time": fake.time(),
        "gender": random.choice([0, 1]),  # 0: 여성, 1: 남성
        "age": random.randint(18, 80),  # 나이
        "type": random.choice(list(type_mapping.keys())),  # 상담 유형 코드
        "call_content": generate_korean_call_content(),  # 한국어 대화 내용
        "negative_count": random.randint(0, 12),  # 부정어 개수
    }
    for _ in range(200)
]

# DataFrame으로 변환
df = pd.DataFrame(data)

# 상담 유형 매핑
df["type_text"] = df["type"].map(type_mapping)

# 점수 및 부정 유형 계산
df["satisfaction_score"] = df["negative_count"].apply(calculate_score)

# 대화 요약 추가
df["call_summary"] = df["call_content"].apply(summarize_call_content)

# 최종 데이터프레임 확인
print("데이터프레임 생성 완료!")
print(df.head())

# CSV 저장
file_path = "processed_january_negative_customers.csv"
df.to_csv(file_path, index=False, encoding="utf-8-sig")
print(f"데이터가 {file_path}에 저장되었습니다.")


데이터프레임 생성 완료!
   customer_id customer_name   product_name  \
0         6984           지중수    롯데손해보험 치아보험   
1         5321           박서준     흥국화재 어린이보험   
2         7629           박춘자     한화생명 어린이보험   
3         9045           이준영    DB손해보험 건강보험   
4         5622           박서현  KB손해보험 변액연금보험   

                                call_id  agent_id   call_date call_time  \
0  5e756162-8787-4f9b-9e7c-42f6c24bf73b       700  2025-01-18  07:10:59   
1  bd0367dd-c26b-44bd-8725-a45951c1d5bc       388  2025-01-22  17:40:14   
2  68276715-9dda-4cd5-a332-8e97b55f8373       984  2025-01-08  16:08:54   
3  2c159eee-8de4-4a1e-ae97-2a4e499ce77b       983  2025-01-14  09:13:05   
4  44f810da-a6bf-4257-a901-29754e435e04       602  2025-01-18  06:06:09   

   gender  age type              call_content  negative_count type_text  \
0       0   66   05           기타 불만 사항이 있습니다.               7      보장내용   
1       1   67   02  보험료를 해지하고 싶은데 절차가 복잡합니다.               1     설계사불만   
2       1   33   04  보험료를 해

In [2]:
#2. 가공데이터에서 1월 기준 불만 고객 테이터추출


# 데이터 로드
df = pd.read_csv(file_path)

# 날짜 형식 변환
df['call_date'] = pd.to_datetime(df['call_date'], errors='coerce')


# 1월 데이터 필터링
january_data = df[df['call_date'].dt.month == 1]

# 점수 기준 부정 이상 필터링
negative_data = january_data[
    january_data['negative_count'] >= 7  # 부정어 개수 기준 7 이상
]

# 결과 확인
print(f"부정 이상 데이터 개수: {negative_data.shape[0]}")
print(negative_data.head())

# 결과 저장
negative_data.to_csv("january_negative_customers.csv", index=False, encoding="utf-8-sig")

부정 이상 데이터 개수: 93
   customer_id customer_name product_name  \
0         6984           지중수  롯데손해보험 치아보험   
2         7629           박춘자   한화생명 어린이보험   
3         9045           이준영  DB손해보험 건강보험   
7         3210           김상훈  NH농협생명 종신보험   
8         6392           주예원    삼성화재 실비보험   

                                call_id  agent_id  call_date call_time  \
0  5e756162-8787-4f9b-9e7c-42f6c24bf73b       700 2025-01-18  07:10:59   
2  68276715-9dda-4cd5-a332-8e97b55f8373       984 2025-01-08  16:08:54   
3  2c159eee-8de4-4a1e-ae97-2a4e499ce77b       983 2025-01-14  09:13:05   
7  4b1dac3f-60cb-42e2-b28a-0c824f759933       116 2025-01-22  15:00:18   
8  8d57a9f8-d692-4e3e-870b-155b3ea08c86       654 2025-01-17  15:30:28   

   gender  age  type              call_content  negative_count type_text  \
0       0   66     5           기타 불만 사항이 있습니다.               7      보장내용   
2       1   33     4  보험료를 해지하고 싶은데 절차가 복잡합니다.              12     보험료해지   
3       1   39     5  보험금을 청구했는데 지급이 지연